In [180]:

import re
import pandas as pd
def prepareEyeTrackingTiming(eyetrackerFile):
    with open(eyetrackerFile, 'r') as file:
        msg_lines = [
            line.strip() for line in file 
            if re.match(r"^MSG\s+\d+", line) and 
               not any(exclude in line for exclude in ["!CAL", "VALIDATE", "!MODE", "ELC", "CAMERA", "THRESHOLDS", "GAZE", "RECC"])
        ]
   

        table = [re.split(r'[\t]+', line) for line in msg_lines]
      
        for i in range(len(table)):
            if i == 0:
                table[i].append(0)  # First row has 0 duration
            else:
                table[i].append(float(table[i][1]) - float(table[i - 1][1]))


        df = pd.DataFrame(table,columns = ["MSG","Time","Event","Type","Status","Duration"])
        moveTime =  df.pop("Time");
        df.pop("MSG")
        df.insert(3, "Time", moveTime)


        print(df)
        print()
        #Iterate through rows and compute things: 
        nextStory = 0
        i = 0
        eyeTrackingLogs = {"story1": pd.DataFrame(),
                           "story2": pd.DataFrame()}
        for j in [0,1]:
            Event, StoryTime, RT, PartDuration = [], [], [], []
            while i <= len(df):
                row = df.iloc[i]
                print(row.to_list())
                ## Start Story 
                if row['Event'] == 'PART' and row['Status'] =='BEG':
                    Event.append("BEGIN")
                    if i == 1 or nextStory:
                        StoryTime.append(float(row['Duration'])/1000)
                        begTime = i
                        print("YAy!")
                        nextStory= 0
                    else:
                        StoryTime.append((float(row['Time'])-float(df.iloc[begTime]['Time'])+ float(df.iloc[begTime]['Duration'])/1000)/1000)
                    RT.append(0)
                    PartDuration.append(0)
                elif row['Event'] == 'PART' and row['Status'] =='END':
                    Event.append("PROBE")
                    StoryTime.append((float(row['Time'])-float(df.iloc[begTime]['Time'])+ float(df.iloc[begTime]['Duration']))/1000)
                    RT.append(0)
                    PartDuration.append(float(row['Duration'])/1000)
                elif row['Event'] =='KEYPRESS':
                    Event.append("KEYPRESS")
                    StoryTime.append((float(row['Time'])-float(df.iloc[begTime]['Time'])+ float(df.iloc[begTime]['Duration']))/1000)
                    RT.append(float(row['Duration'])/1000)
                    PartDuration.append(float(df.iloc[i-1]['Duration'])/1000)
                elif row['Event'] == 'RECALL': #Break on the nearest recall
                    i +=2
                    nextStory = 1
                    break;
                i +=1;

            eyeTrackingLogs[list(eyeTrackingLogs.keys())[j]] = pd.DataFrame(list(map(list, zip(*[Event,StoryTime,RT,PartDuration]))),columns=["Event","StoryTime","RT","partDuration"])
            print(eyeTrackingLogs[list(eyeTrackingLogs.keys())[j]] )

        return eyeTrackingLogs
    

def compareEyeTrackingWithBeh(eyeTrackingLogs,logStory1,logStory2):

    ### Print Differences 
    df_diff1 = eyeTrackingLogs['story1'] .copy()
    df_diff1[["StoryTime", "RT", "partDuration"]] = eyeTrackingLogs['story1'][["StoryTime", "RT", "partDuration"]] - logStory1[["StoryTime", "RT", "partDuration"]]
    print(df_diff1)


    df_diff2 = eyeTrackingLogs['story2'] .copy()
    df_diff2[["StoryTime", "RT", "partDuration"]] = eyeTrackingLogs['story2'][["StoryTime", "RT", "partDuration"]] - logStory2[["StoryTime", "RT", "partDuration"]]
    print(df_diff2)

    ## get Numeric data: 
    numDiffData1 = df_diff1.select_dtypes(include='number').to_numpy()
    numDiffData2 = df_diff2.select_dtypes(include='number').to_numpy()
    def diffr(numDiffData,df_diff):
        for i in range(numDiffData.shape[1]):
            for j in range(numDiffData.shape[0]):
                if numDiffData[j,i] > 0.005:
                    print(f"WARNING! Exceeding 5ms delay threshold between EyeLink and behavior in row {j} ({df_diff.iloc[j]['Event']}) on {df_diff.columns.values.tolist()[i]} (Delay: {numDiffData1[j,i]:.5f})")
                else:
                    print(f"Delay is in norm for row {j} ({df_diff.iloc[j]['Event']}) on {df_diff.columns.values.tolist()[i]} (Delay: {numDiffData[j,i]:.5f})")

    print("\n Differnce in Story 1 Check: ")
    diffr(numDiffData1,df_diff1)
    print("\n Differnce in Story 2 Check: ")
    diffr(numDiffData2,df_diff2)

In [ ]:
import os,pickle

data1 = {
    "Event": ["BEGIN", "PROBE", "KEYPRESS"],
    "StoryTime": [0.007510, 35.294762, 37.026710],
    "RT": [0.00000, 0.00000, 1.73195],
    "partDuration": [0.000000, 35.284102, 35.284102]
}

data2 = {
    "Event": ["BEGIN", "PROBE", "KEYPRESS"],
    "StoryTime": [0.007211, 40.539256, 41.973531],
    "RT": [0.000000, 0.000000, 1.434277],
    "partDuration": [0.000000, 40.528899, 40.528899]
}

log_story1 = pd.DataFrame(data1)
log_story2 = pd.DataFrame(data2)
subject = "mich7"
path = "C:\\Users\\Lab\Documents\\StrategicMW_lite\\Experiment\\data"
EyetrackerDF = prepareEyeTrackingTiming(os.path.join(path,subject,subject+".asc"))
with open(os.path.join(path,subject,subject+".pickle"), 'rb') as handle:
    behavioralDatab = pickle.load(handle)

log_story1 = behavioralDatab['story1_beh']
log_story2 = behavioralDatab['story2_beh']
compareEyeTrackingWithBeh(EyetrackerDF,log_story1,log_story2)

        Event          Type Status      Time  Duration
0      LISTEN       STORY_1    BEG   922269        0.0
1        PART  INTRODUCTION    BEG   922276        7.0
2        PART  INTRODUCTION    END   957566    35290.0
3    KEYPRESS           100      0   959203     1637.0
4      LISTEN       STORY_1    END   959203        0.0
5      RECALL       STORY_1    BEG   959207        4.0
6      RECALL       STORY_1    END   974617    15410.0
7       START       STORY_2    BEG   980912     6295.0
8       START       STORY_2    END   982225     1313.0
9      LISTEN       STORY_2    BEG   982341      116.0
10       PART  INTRODUCTION    BEG   982349        8.0
11       PART  INTRODUCTION    END  1022875    40526.0
12   KEYPRESS           100      0  1027985     5110.0
13     LISTEN       STORY_2    END  1027985        0.0
14     RECALL       STORY_2    BEG  1027988        3.0
15     RECALL       STORY_2    END  1045271    17283.0
16  PROCEDURE             0    END  1045273        2.0

['LISTEN'

In [32]:
log_story1

,Event,StoryTime,RT,partDuration
0,BEGIN,0.007510,0.00000,0.000000
1,PROBE,35.294762,0.00000,35.284102
2,KEYPRESS,37.026710,1.73195,35.284102
